In [78]:
import copernicusmarine as cm
import xarray as xr
import polars as pl
import numpy as np
from datetime import datetime
import sys
import os
sys.path.append(os.path.abspath("../python/utils"))
from get_data import make_request_and_open_copernicus

In [79]:
temp = pl.read_csv("../../data/temporal_subset.csv")
box = pl.read_csv("../../data/world_box.csv")

In [80]:
day = temp.filter(pl.col("mas_actual") == 0)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 0)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 0)["year"].to_list()[0]
min_time = f"{day}-{month}-{year}"
day = temp.filter(pl.col("mas_actual") == 1)["day"].to_list()[0]
month = temp.filter(pl.col("mas_actual") == 1)["month"].to_list()[0]
year = temp.filter(pl.col("mas_actual") == 1)["year"].to_list()[0]
max_time = f"{day}-{month}-{year}"
fecha_min = datetime.strptime(min_time, "%d-%m-%Y")
fecha_min_format = fecha_min.strftime("%Y-%m-%dT%H:%M:%S")
fecha_max = datetime.strptime(max_time, "%d-%m-%Y")
fecha_max_format = fecha_max.strftime("%Y-%m-%dT%H:%M:%S")
print("min", fecha_min_format)
print("max", fecha_max_format)

min 1999-01-01T00:00:00
max 2013-12-24T00:00:00


## Fecha mínima provisional

Para trabajar con un archivo más manejable, usaremos un subset más pequeño y luego al final usaremos el completo.

In [81]:
fecha_min_provisional = "24-12-2012"
fecha_min_provisional = datetime.strptime(fecha_min_provisional, "%d-%m-%Y")
fecha_min_format = fecha_min_provisional.strftime("%Y-%m-%dT%H:%M:%S")

In [82]:
max_latitude = box["max_latitude"].to_list()[0]
min_latitude = box["min_latitude"].to_list()[0]
max_longitude = box["max_longitude"].to_list()[0]
min_longitude = box["min_longitude"].to_list()[0]
print("max", max_latitude, max_longitude)
print("min", min_latitude, min_longitude)

max -43.794483 89.186795
min -65.655159 -61.236411


## Request

In [83]:
ds = make_request_and_open_copernicus("cmems_mod_glo_phy_my_0.083deg_P1D-m", "Global_Ocean_Physics_Reanalysis.nc",
                                        fecha_min_format, fecha_max_format, max_latitude, min_latitude, max_longitude, min_longitude)
ds

INFO - 2025-03-21T11:59:28Z - Downloading Copernicus Marine data requires a Copernicus Marine username and password, sign up for free at: https://data.marine.copernicus.eu/register


Copernicus Marine username:Copernicus Marine password:

INFO - 2025-03-21T11:59:39Z - Selected dataset version: "202311"
INFO - 2025-03-21T11:59:39Z - Selected dataset part: "default"
WARNING - 2025-03-21T11:59:44Z - Some of your subset selection [0, 1] for the depth dimension exceed the dataset coordinates [0.49402499198913574, 5727.9169921875]
INFO - 2025-03-21T11:59:50Z - Starting download. Please wait...
100%|██████████| 8385/8385 [12:30<00:00, 11.18it/s]  
INFO - 2025-03-21T12:12:21Z - Successfully downloaded to /home/pablo/Desktop/zird/2/proy/github/data/copernicus/Global_Ocean_Physics_Reanalysis_(2).nc


<xarray.Dataset> Size: 83MB
Dimensions:    (depth: 1, latitude: 262, longitude: 1805, time: 2)
Coordinates:
  * depth      (depth) float32 4B 0.494
  * latitude   (latitude) float32 1kB -65.58 -65.5 -65.42 ... -43.92 -43.83
  * longitude  (longitude) float32 7kB -61.17 -61.08 -61.0 ... 89.0 89.08 89.17
  * time       (time) datetime64[ns] 16B 2013-12-23 2013-12-24
Data variables:
    bottomT    (time, latitude, longitude) float64 8MB ...
    mlotst     (time, latitude, longitude) float64 8MB ...
    siconc     (time, latitude, longitude) float64 8MB ...
    sithick    (time, latitude, longitude) float64 8MB ...
    so         (time, depth, latitude, longitude) float64 8MB ...
    thetao     (time, depth, latitude, longitude) float64 8MB ...
    uo         (time, depth, latitude, longitude) float64 8MB ...
    usi        (time, latitude, longitude) float64 8MB ...
    vo         (time, depth, latitude, longitude) float64 8MB ...
    vsi        (time, latitude, longitude) float64 8MB ...
    zos        (time, latitude, longitude) float64 8MB ...
Attributes:
    title:                     daily mean fields from Global Ocean Physics An...
    references:                http://www.mercator-ocean.fr
    comment:                   CMEMS product
    Conventions:               CF-1.4
    institution:               MERCATOR OCEAN
    history:                   2023/06/01 16:20:05 MERCATOR OCEAN Netcdf crea...
    source:                    MERCATOR GLORYS12V1
    copernicusmarine_version:  2.0.1

In [86]:
lat_bins = np.arange(ds.latitude.min(), ds.latitude.max() + 1, 1)
lon_bins = np.arange(ds.longitude.min(), ds.longitude.max() + 1, 1)

lat_labels = (lat_bins[:-1] + lat_bins[1:]) / 2
lon_labels = (lon_bins[:-1] + lon_bins[1:]) / 2

ds_binned = ds.groupby_bins("latitude", lat_bins, labels=lat_labels).mean()
ds_binned = ds_binned.groupby_bins("longitude", lon_bins, labels=lon_labels).mean()
print(ds_binned)

<xarray.Dataset> Size: 586kB
Dimensions:         (longitude_bins: 151, latitude_bins: 22, time: 2, depth: 1)
Coordinates:
  * depth           (depth) float32 4B 0.494
  * time            (time) datetime64[ns] 16B 2013-12-23 2013-12-24
  * latitude_bins   (latitude_bins) float64 176B -65.08 -64.08 ... -45.08 -44.08
  * longitude_bins  (longitude_bins) float64 1kB -60.67 -59.67 ... 88.33 89.33
Data variables:
    bottomT         (longitude_bins, latitude_bins, time) float64 53kB -1.562...
    mlotst          (longitude_bins, latitude_bins, time) float64 53kB 10.53 ...
    siconc          (longitude_bins, latitude_bins, time) float64 53kB 0.9659...
    sithick         (longitude_bins, latitude_bins, time) float64 53kB 1.48 ....
    so              (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    thetao          (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    uo              (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    usi            

In [87]:
ds_binned

<xarray.Dataset> Size: 586kB
Dimensions:         (longitude_bins: 151, latitude_bins: 22, time: 2, depth: 1)
Coordinates:
  * depth           (depth) float32 4B 0.494
  * time            (time) datetime64[ns] 16B 2013-12-23 2013-12-24
  * latitude_bins   (latitude_bins) float64 176B -65.08 -64.08 ... -45.08 -44.08
  * longitude_bins  (longitude_bins) float64 1kB -60.67 -59.67 ... 88.33 89.33
Data variables:
    bottomT         (longitude_bins, latitude_bins, time) float64 53kB -1.562...
    mlotst          (longitude_bins, latitude_bins, time) float64 53kB 10.53 ...
    siconc          (longitude_bins, latitude_bins, time) float64 53kB 0.9659...
    sithick         (longitude_bins, latitude_bins, time) float64 53kB 1.48 ....
    so              (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    thetao          (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    uo              (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    usi             (longitude_bins, latitude_bins, time) float64 53kB 7.63e-...
    vo              (longitude_bins, latitude_bins, time, depth) float64 53kB ...
    vsi             (longitude_bins, latitude_bins, time) float64 53kB -7.63e...
    zos             (longitude_bins, latitude_bins, time) float64 53kB -1.701...
Attributes:
    title:                     daily mean fields from Global Ocean Physics An...
    references:                http://www.mercator-ocean.fr
    comment:                   CMEMS product
    Conventions:               CF-1.4
    institution:               MERCATOR OCEAN
    history:                   2023/06/01 16:20:05 MERCATOR OCEAN Netcdf crea...
    source:                    MERCATOR GLORYS12V1
    copernicusmarine_version:  2.0.1

In [90]:
ds_binned.bottomT.size

6644